In [5]:
!pip install spacy
!python -m spacy download es_core_news_sm
!pip install pdfplumber --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 77.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 124.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 285.2 MB/s eta 0:00:00


In [1]:
import pdfplumber  # Para extraer texto de PDFs
import re  # Para trabajar con expresiones regulares
import spacy  # Para procesamiento de lenguaje natural (NLP)
import unicodedata  # Para normalizar caracteres Unicode

In [2]:
nlp = spacy.load("es_core_news_sm")  #cargar el modelo en español

In [3]:
pdf_path = "../Archivos/certificado.pdf"

In [4]:
header_pattern = r"Cámara de Comercio de Barranquilla\s*CERTIFICADO DE EXISTENCIA Y REPRESENTACION LEGAL O\s*DE INSCRIPCION DE DOCUMENTOS\.\s*Fecha de expedición:.*?\nRecibo No\..*?\nCODIGO DE VERIFICACIÓN:.*?\n"

In [5]:
def extract_text_without_header(pdf_path):
    extracted_text = []  # Lista donde almacenaremos el texto de cada página

    with pdfplumber.open(pdf_path) as pdf:  # Abrir el archivo PDF
        for page in pdf.pages:  # Iterar por cada página del PDF
            text = page.extract_text()  # Extraer texto de la página
            if text:  # Verificar si hay texto en la página
                text = re.sub(header_pattern, "", text, flags=re.DOTALL)  # Eliminar encabezado con regex
                extracted_text.append(text)  # Guardar el texto limpio en la lista

    return " ".join(extracted_text)  # Unir todas las páginas en un solo texto

In [6]:
def preprocess_text(text):
    text = unicodedata.normalize("NFKD", text)  # Normalizar caracteres Unicode (ej. á → a)

    doc = nlp(text.lower())  # Convertir todo el texto a minúsculas y procesarlo con spaCy

    tokens = [token.text for token in doc if token.is_alpha and not token.is_stop]
    return tokens

In [7]:
raw_text = extract_text_without_header(pdf_path)  # Extraer texto limpio del PDF
tokens = preprocess_text(raw_text)  # Preprocesar el texto con spaCy

print(tokens[:50])  # Mostrar los primeros 50 tokens como prueba

['verifique', 'contenido', 'confiabilidad', 'certificado', 'ingresando', 'digite', 'visualice', 'imagen', 'generada', 'momento', 'ilimitada', 'calendario', 'contados', 'fecha', 'atencion', 'comerciante', 'cumplido', 'deber', 'legal', 'renovar', 'matricula', 'mercantil', 'd', 's', 'matricula', 'mercantil', 'proporciona', 'seguridad', 'confianza', 'ean', 'n', 'renueve', 'matricula', 'mercantil', 'tardar', 'marzo', 'v', 'i', 'c', 'n', 'n', 's', 'fundamento', 'inscrripciones', 'efectu', 'adas', 'registro', 'mercantil', 'comercio', 'certiefica']


In [8]:
with open("../Resultado/certificado_limpio.txt", "w", encoding="utf-8") as f:
    f.write(" ".join(tokens))